##  Import Libraries

In [ ]:
import requests
import pandas as pd
import time
from sqlalchemy import create_engine
import os
from bs4 import BeautifulSoup
import csv
import mysql.connector
import getpass
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sw

# Data Aquisition

https://data.sandiego.gov/datasets/crb-cases/

In [ ]:
# Set the URL for the page with the download links
url = 'https://data.sandiego.gov/datasets/crb-cases/'

# Set the folder path to store the CSV files
folder_path = 'C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder'   # Replace with the desired folder path

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Fetch the HTML content
response = requests.get(url, verify=False)
if response.status_code == 200:
    html_content = response.text

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all <paper-button> elements with the class "resource-table-button"
    button_elements = soup.find_all('paper-button', class_='resource-table-button')

    # Extract the download links
    download_links = []
    for button in button_elements:
        # Get the "property" attribute value
        property_value = button.get('property')

        # Check if it's the download link button
        if property_value == 'dcat:accessURL':
            # Get the URL from the parent <a> tag
            url = button.parent['href']
            download_links.append(url)

    # Download and save each CSV file
    for link in download_links:
        # Set the filename based on the last part of the URL
        file_name = link.split('/')[-1]

        # Fetch the CSV data
        response = requests.get(link)
        if response.status_code == 200:
            csv_data = response.content

            # Save the CSV file to the folder
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'wb') as file:
                file.write(csv_data)
            print(f"CSV file saved to: {file_path}")
        else:
            print(f"Failed to fetch the CSV file: {link}")
else:
    print("Failed to fetch the HTML content.")


CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_cases_fy2022_datasd.csv
CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_allegations_fy2022_datasd.csv
CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_cases_bwc_fy2022_datasd.csv
CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_cases_fy2021_datasd.csv
CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_allegations_fy2021_datasd.csv
CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_cases_bwc_fy2021_datasd.csv
CSV file saved to: C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder\crb_cases_fy2020_datasd.csv


In [ ]:
# Set the MySQL database connection details
host = 'localhost'  # Replace with your MySQL host
user = 'root'  # Replace with your MySQL username
password = getpass.getpass(prompt='Password:',stream=None)  # Replace with your MySQL password

In [ ]:
# Set the folder path containing the CSV files
folder_path = 'C:/Users/earne/OneDrive - University of San Diego/MSADS/ADS-599/git/ADS599_Team1/Data Folder'  # Replace with the folder path where CSV files are stored

# Create a MySQL database connection
cnx = mysql.connector.connect(host=host, user=user, password=password)
cursor = cnx.cursor()

# Create the 'crb' database if it doesn't exist
create_database_query = 'CREATE DATABASE IF NOT EXISTS crb'
cursor.execute(create_database_query)

In [ ]:
# Use the 'crb' database
use_database_query = 'USE crb'
cursor.execute(use_database_query)

In [ ]:
# Fetch the list of CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Iterate over each CSV file and create a table in the 'crb' database
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    table_name = csv_file[:-4]  # Remove the '.csv' extension to use it as the table name

    # Read the CSV file
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)

        # Read the header row to extract the column names
        header_row = next(csv_reader)
        column_names = [col.strip() for col in header_row]  # Remove leading/trailing spaces

        # Initialize a dictionary to store the maximum lengths for each column
        max_lengths = {col: 0 for col in column_names}

        # Iterate over each row to determine the maximum length for each column
        for row in csv_reader:
            for col, value in zip(column_names, row):
                value_length = len(value.strip())  # Get the length of the value without leading/trailing spaces
                max_lengths[col] = max(max_lengths[col], value_length)

        # Generate the CREATE TABLE query with the appropriate column lengths
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ('
        for col in column_names:
            column_length = max_lengths[col] + 10  # Add additional buffer to the maximum length
            create_table_query += f'`{col}` VARCHAR({column_length}), '
        create_table_query = create_table_query.rstrip(', ') + ')'

        # Create the table in the 'crb' database
        cursor.execute(create_table_query)

        # Reset the file pointer to the beginning of the CSV file
        file.seek(0)

        # Skip the header row
        next(csv_reader)

        # Iterate over each row and insert into the table
        for row in csv_reader:
            insert_query = f'INSERT INTO `{table_name}` VALUES ({",".join("%s" for _ in row)})'
            values = [value.strip() for value in row]

            cursor.execute(insert_query, values)

# Commit the changes and close the database connection
cnx.commit()
cursor.close()

NameError: name 'os' is not defined

In [ ]:
tables = pd.DataFrame(pd.read_sql('Show Tables', cnx))
tables

,Tables_in_crb
0,allyrs
1,crb_allegations_fy2020_datasd
2,crb_allegations_fy2021_datasd
3,crb_allegations_fy2022_datasd
4,crb_cases_bwc_fy19_datasd
5,crb_cases_bwc_fy2020_datasd
6,crb_cases_bwc_fy2021_datasd
7,crb_cases_bwc_fy2022_datasd
8,crb_cases_fy19_datasd
9,crb_cases_fy2020_datasd


The dictionary explains what each of the variables are in the data sets.  We'll pull the list from the database here.

In [ ]:
dict_df = pd.DataFrame(pd.read_sql('SELECT * FROM dictionary', cnx))
dict_df

,﻿field,description,possible_values
0,id,Case id,
1,case_number,Case number,
2,team,CRB Team number,
3,date_assigned,Date assigned to CRB,
4,date_completed,Date CRB team completed case,
5,date_presented,Date CRB deliberated on case,
6,days_number,,
7,days_60_or_less,Number of days for case to be reviewed,
8,days_90_or_less,Number of days for case to be reviewed,
9,days_120_or_less,Number of days for case to be reviewed,


For our effort, we are most interested in any case specific data.  So we will do a query that pulls the id, case_number, team, allegation, ia_finding, crb_decision, changes, unanimous_vote, pd_division, crb_viewed_bwc, complainant_race, and complainant_gender columns from the data.  We'll start with doing this by year.

Note: fy19 is the first year of reporting.  The data here is combined into one table, so no joining is required.

In [ ]:
fy19 = pd.DataFrame(pd.read_sql("""SELECT crb_cases_fy19_datasd.id, 
                                crb_cases_fy19_datasd.case_number,
                                crb_cases_fy19_datasd.allegation,
                                crb_cases_fy19_datasd.ia_finding,
                                crb_cases_fy19_datasd.crb_decision,
                                crb_cases_fy19_datasd.unanimous_vote,
                                crb_cases_fy19_datasd.date_assigned,
                                crb_Cases_fy19_datasd.date_presented,
                                crb_cases_fy19_datasd.pd_division,
                                crb_cases_fy19_datasd.complainant_race,
                                crb_cases_fy19_datasd.complainant_gender,
                                crb_cases_fy19_datasd.changes
                                FROM crb_cases_fy19_datasd
                                ;""", cnx))

fy19.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,complainant_race,complainant_gender,changes
0,19-59,2018-0182,ARREST,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO
1,19-59,2018-0182,FORCE,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO
2,19-59,2018-0182,OF1 PROCEDURE,S,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO
3,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO
4,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO


In [ ]:
fy20 = pd.DataFrame(pd.read_sql("""SELECT crb_allegations_fy2020_datasd.id, 
                                crb_allegations_fy2020_datasd.case_number,
                                crb_allegations_fy2020_datasd.allegation,
                                crb_allegations_fy2020_datasd.ia_finding,
                                crb_allegations_fy2020_datasd.crb_decision,
                                crb_allegations_fy2020_datasd.unanimous_vote,
                                crb_cases_fy2020_datasd.date_assigned,
                                crb_Cases_fy2020_datasd.date_presented,
                                crb_cases_fy2020_datasd.pd_division,
                                crb_cases_fy2020_datasd.crb_viewed_bwc,
                                crb_cases_fy2020_datasd.complainant_race,
                                crb_cases_fy2020_datasd.complainant_gender,
                                crb_cases_fy2020_datasd.changes
                                FROM crb_allegations_fy2020_datasd
                                JOIN crb_cases_fy2020_datasd ON crb_allegations_fy2020_datasd.id = crb_cases_fy2020_datasd.id
                                ;""", cnx))

fy20.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,crb_viewed_bwc,complainant_race,complainant_gender,changes
0,20-29,2019-0130,OIS,WP,A,YES,2019-11-05,2020-03-10,SOUTHEASTERN,YES,W,M,NO
1,20-29,2019-0130,OIS,WP,A,YES,2019-11-05,2020-03-10,SOUTHEASTERN,YES,W,M,NO
2,20-29,2019-0130,OIS,WP,A,YES,2019-11-05,2020-03-10,SOUTHEASTERN,YES,W,M,NO
3,20-29,2019-0130,OF1-PROCEDURE,S,A,YES,2019-11-05,2020-03-10,SOUTHEASTERN,YES,W,M,NO
4,20-29,2019-0130,OIS,WP,A,YES,2019-11-05,2020-03-10,SOUTHEASTERN,YES,W,M,NO


In [ ]:
fy21 = pd.DataFrame(pd.read_sql("""SELECT crb_allegations_fy2021_datasd.id, 
                                crb_allegations_fy2021_datasd.case_number,
                                crb_allegations_fy2021_datasd.allegation,
                                crb_allegations_fy2021_datasd.ia_finding,
                                crb_allegations_fy2021_datasd.crb_decision,
                                crb_allegations_fy2021_datasd.unanimous_vote,
                                crb_cases_fy2021_datasd.date_assigned,
                                crb_Cases_fy2021_datasd.date_presented,
                                crb_cases_fy2021_datasd.pd_division,
                                crb_cases_fy2021_datasd.crb_viewed_bwc,
                                crb_cases_fy2021_datasd.complainant_race,
                                crb_cases_fy2021_datasd.complainant_gender,
                                crb_cases_fy2021_datasd.changes
                                FROM crb_allegations_fy2021_datasd
                                JOIN crb_cases_fy2021_datasd ON crb_allegations_fy2021_datasd.id = crb_cases_fy2021_datasd.id
                                ;""", cnx))

fy21.head()


,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,crb_viewed_bwc,complainant_race,complainant_gender,changes
0,21-01,2019-0671,FORCE,S,A,YES,2020-05-06,2020-07-28,MID-CITY,YES,B,M,NO
1,21-02,2019-0617,OF2 PROCEDURE,S,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO
2,21-02,2019-0617,OF3 PROCEDURE,S,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO
3,21-02,2019-0617,FORCE,U,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO
4,21-02,2019-0617,FORCE,U,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO


In [ ]:
fy22 = pd.DataFrame(pd.read_sql("""SELECT crb_allegations_fy2022_datasd.id, 
                                crb_allegations_fy2022_datasd.case_number,
                                crb_allegations_fy2022_datasd.allegation,
                                crb_allegations_fy2022_datasd.ia_finding,
                                crb_allegations_fy2022_datasd.crb_decision,
                                crb_allegations_fy2022_datasd.unanimous_vote,
                                crb_cases_fy2022_datasd.date_assigned,
                                crb_cases_fy2022_datasd.date_presented,
                                crb_cases_fy2022_datasd.pd_division,
                                crb_cases_fy2022_datasd.crb_viewed_bwc,
                                crb_cases_fy2022_datasd.complainant_race,
                                crb_cases_fy2022_datasd.complainant_gender,
                                crb_cases_fy2022_datasd.changes
                                FROM crb_allegations_fy2022_datasd
                                JOIN crb_cases_fy2022_datasd ON crb_allegations_fy2022_datasd.id = crb_cases_fy2022_datasd.id
                                ;""", cnx))

fy22.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,crb_viewed_bwc,complainant_race,complainant_gender,changes
0,22-01,2019-0241,PROCEDURE,NS,A,YES,2019-09-05,2021-07-13,SOUTHEASTERN,YES,B,M,YES
1,22-01,2019-0241,PROCEDURE,U,A,YES,2019-09-05,2021-07-13,SOUTHEASTERN,YES,B,M,YES
2,22-01,2019-0241,FORCE,E,A/C,YES,2019-09-05,2021-07-13,SOUTHEASTERN,YES,B,M,YES
3,22-01,2019-0241,ARREST,E,A,YES,2019-09-05,2021-07-13,SOUTHEASTERN,YES,B,M,YES
4,22-02,2020-0576,DETENTION,E,A/C,YES,2021-05-14,2021-07-13,SOUTHEASTERN,YES,B,M,NO


In [ ]:
allyrs = pd.concat([fy19, fy20, fy21, fy22], axis=0)

allyrs = allyrs.reset_index(drop=True)
allyrs.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,complainant_race,complainant_gender,changes,crb_viewed_bwc
0,19-59,2018-0182,ARREST,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NaN
1,19-59,2018-0182,FORCE,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NaN
2,19-59,2018-0182,OF1 PROCEDURE,S,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NaN
3,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO,NaN
4,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO,NaN


# Data Cleaning

Now that we've pulled data for each year from our database and combined into one database, we'll start processing and cleaning the data

In [ ]:
allyrs.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,complainant_race,complainant_gender,changes,crb_viewed_bwc
0,19-59,2018-0182,ARREST,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NaN
1,19-59,2018-0182,FORCE,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NaN
2,19-59,2018-0182,OF1 PROCEDURE,S,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NaN
3,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO,NaN
4,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO,NaN


In [ ]:
fy21.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,crb_viewed_bwc,complainant_race,complainant_gender,changes
0,21-01,2019-0671,FORCE,S,A,YES,2020-05-06,2020-07-28,MID-CITY,YES,B,M,NO
1,21-02,2019-0617,OF2 PROCEDURE,S,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO
2,21-02,2019-0617,OF3 PROCEDURE,S,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO
3,21-02,2019-0617,FORCE,U,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO
4,21-02,2019-0617,FORCE,U,A,YES,2020-04-23,2020-07-28,NATIONAL CITY Special ops,YES,,F,NO


In [ ]:
#Inspect the dataframe
allyrs.dtypes

id                    object
case_number           object
allegation            object
ia_finding            object
crb_decision          object
unanimous_vote        object
date_assigned         object
date_presented        object
pd_division           object
complainant_race      object
complainant_gender    object
changes               object
crb_viewed_bwc        object
dtype: object

In [ ]:
#Total number of cases from 2019 to 2022
len(allyrs)

1947

In [ ]:
allyrs.isna().sum()

id                      0
case_number             0
allegation              0
ia_finding              0
crb_decision            0
unanimous_vote          0
date_assigned           0
date_presented          0
pd_division             0
complainant_race        0
complainant_gender      0
changes                 0
crb_viewed_bwc        213
dtype: int64

In [ ]:
# replace na values in allyrs with 'NO'
allyrs['crb_viewed_bwc'] = allyrs['crb_viewed_bwc'].fillna('NO')
allyrs.isna().sum()

id                    0
case_number           0
allegation            0
ia_finding            0
crb_decision          0
unanimous_vote        0
date_assigned         0
date_presented        0
pd_division           0
complainant_race      0
complainant_gender    0
changes               0
crb_viewed_bwc        0
dtype: int64

In [ ]:
dfs = [fy19, fy20, fy21, fy22, allyrs]

# List of date column names
date_columns = ['date_assigned', 'date_presented']

# Iterate over each DataFrame
for df in dfs:
    # Convert date columns to datetime with "YYYY-MM-DD" format
    df[date_columns] = df[date_columns].apply(pd.to_datetime, format='%Y-%m-%d')



ValueError: time data "1/29/19" doesn't match format "%Y-%m-%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
fy19.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,complainant_race,complainant_gender,changes
0,19-59,2018-0182,ARREST,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO
1,19-59,2018-0182,FORCE,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO
2,19-59,2018-0182,OF1 PROCEDURE,S,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO
3,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO
4,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO


In [ ]:
allyrs.head()

,id,case_number,allegation,ia_finding,crb_decision,unanimous_vote,date_assigned,date_presented,pd_division,complainant_race,complainant_gender,changes,crb_viewed_bwc
0,19-59,2018-0182,ARREST,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NO
1,19-59,2018-0182,FORCE,E,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NO
2,19-59,2018-0182,OF1 PROCEDURE,S,A,YES,1/29/19,6/11/19,CENTRAL,B,M,NO,NO
3,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO,NO
4,19-58,2019-0152,DISCRIMINATION,U,A,YES,4/17/19,6/25/19,WESTERN,W,M,NO,NO


In [ ]:
allyrs.shape

(1947, 13)

In [ ]:
allyrs.dtypes

id                    object
case_number           object
allegation            object
ia_finding            object
crb_decision          object
unanimous_vote        object
date_assigned         object
date_presented        object
pd_division           object
complainant_race      object
complainant_gender    object
changes               object
crb_viewed_bwc        object
dtype: object

In [ ]:
#missing values
allyrs.isna().sum()

id                    0
case_number           0
allegation            0
ia_finding            0
crb_decision          0
unanimous_vote        0
date_assigned         0
date_presented        0
pd_division           0
complainant_race      0
complainant_gender    0
changes               0
crb_viewed_bwc        0
dtype: int64

In [ ]:
#check for issues in the gender and race columns
allyrs['complainant_gender'].value_counts()

complainant_gender
M      1092
F       802
UNK      22
F,M      13
M,M       7
          7
F,F       3
MM        1
Name: count, dtype: int64

In [ ]:
for df in dfs:
    df['complainant_gender'] = df['complainant_gender'].replace({'M,M': 'M', 'MM': 'M', 'F,F': 'F', '': 'UNK',
                                                                     'F,M': 'UNK'})

allyrs['complainant_gender'].value_counts()

complainant_gender
M      1100
F       805
UNK      42
Name: count, dtype: int64

In [ ]:
allyrs['complainant_race'].value_counts()

complainant_race
B                           859
H                           448
W                           366
O                            82
UNK                          72
unkn                         41
                             27
A                            10
H,H                           7
W,B                           7
B,B                           6
ME                            5
S                             4
F                             4
NOT STATED IN THE REPORT      3
W,W                           3
P                             2
HH                            1
Name: count, dtype: int64

In [ ]:
for df in dfs:
    df['complainant_race'] = df['complainant_race'].replace({'H,H': 'H', 'HH': 'H', 'B,B': 'B', 'W,W': 'W',
                                                                 'unkn': 'UNK', 'W,B': 'UNK', 'ME': 'UNK', 'NOT STATED IN THE REPORT': 'UNK',
                                                                 '': 'UNK'})

allyrs['complainant_race'].value_counts()

complainant_race
B      865
H      456
W      369
UNK    155
O       82
A       10
F        4
S        4
P        2
Name: count, dtype: int64

In [ ]:
allyrs['crb_decision'].value_counts()

crb_decision
A      1730
A/C     189
D/C      25
AC        3
Name: count, dtype: int64

In [ ]:
allyrs['crb_viewed_bwc'].value_counts()

crb_viewed_bwc
YES                             1679
NO                               228
NO (OTHER VIDEO)                  22
NO (DELETED RETENTION)             6
                                   4
NO - NOT ENFORCEMENT CONTACT       4
OTHER VIDEO                        3
N/A                                1
Name: count, dtype: int64

In [ ]:
allyrs['crb_viewed_bwc'] = allyrs['crb_viewed_bwc'].replace({'NO (OTHER VIDEO)': 'NO', 'NO (DELETED RETENTION)': 'NO',
                                                              'NO - NOT ENFORCEMENT CONTACT': 'NO', 'N/A': 'NO', 'OTHER VIDEO': 'NO', '': 'NO'})

allyrs['crb_viewed_bwc'].value_counts()

crb_viewed_bwc
YES    1679
NO      268
Name: count, dtype: int64

In [ ]:
allyrs['changes'].value_counts()

changes
NO     1425
YES     492
         30
Name: count, dtype: int64